In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.naive_bayes import MultinomialNB
import nltk
import matplotlib
from matplotlib import pyplot as plt

In [2]:
train=pd.read_csv('train.csv')

In [3]:
train_topic=train[['user.id','topic']]
train_topic=pd.concat([train_topic,pd.get_dummies(train_topic['topic'])],axis=1)
train_topic=train_topic.groupby('user.id').sum().reset_index()

In [4]:
train_id=train_topic['user.id']

In [5]:
train_count=train.groupby(['user.id'])["user.id"].size().reset_index(name='Count')
train_topic=pd.merge(train_topic,train_count,how='left',on='user.id')
train_merge=train_topic.iloc[:,1:41].div(train_topic.iloc[:,-1],axis=0)
train_merge=train_merge.iloc[:,0:40].div(1000,axis=0)
train_merge.head()

,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,BusinessServices,Chemicals,...,RealEstate,Religion,Science,Sports-Recreation,Student,Technology,Telecommunications,Tourism,Transportation,indUnk
0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.001
1,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.001
2,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.001,0.0,0.0,0.0,0.0,0.000
3,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000
4,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.001,0.0,0.0,0.0,0.0,0.000


In [6]:
train_count=train_count['Count']
train_count=train_count.div(train_count.max(),axis=0)
train_count=train_count.div(1000,axis=0)

0    1.347240e-05
1    3.476749e-06
2    4.345937e-07
3    9.561060e-06
4    2.877010e-04
Name: Count, dtype: float64

In [7]:
train_merge=train_merge.join(train_id,how='right')
train_merge=train_merge.join(train_count,how='left')

In [8]:
train=train.groupby(['user.id','age'])["text"].apply(lambda x: "%s" % ' '.join(x)).reset_index(name='text')
train=pd.merge(train,train_merge,how='left',on='user.id')

In [9]:
train.head(10)

,user.id,age,text,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,...,Religion,Science,Sports-Recreation,Student,Technology,Telecommunications,Tourism,Transportation,indUnk,Count
0,1,17,Doritos Fuel of Space urlLink : Ian My...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,1.347240e-05
1,3,16,"Yeah, I set up this blog so Heather w...",0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,3.476749e-06
2,4,17,Not too much exciting has happened ...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001,0.0,0.000,0.0,0.0,0.000,4.345937e-07
3,5,47,European Pilgrimage For High sch...,0.001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,9.561060e-06
4,6,16,Last night's dream is very blurry. T...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001,0.0,0.000,0.0,0.0,0.000,2.877010e-04
5,7,25,"June 21, 2004 SPAIN V P...",0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,1.303781e-06
6,12,23,"Well, 1 day left... Less than 36 ...",0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,4.345937e-06
7,14,37,Every now and then you see a post on one of...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.001,0.0,0.0,0.000,4.172099e-05
8,15,17,Well around 3 me adam and jake went ou...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,1.173403e-05
9,16,15,"yesterday,went out wif my sis...",0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001,0.0,0.000,0.0,0.0,0.000,1.477618e-05


In [10]:
nltk.download('stopwords')
sw = stopwords.words('english')
def stopwords(text):
    '''a function for removing the stopword'''
    # removing the stop words and lowercasing the selected words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)
train['text'] = train['text'].apply(stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
stemmer = SnowballStemmer("english")

def stemming(text):    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text) 
train['text'] = train['text'].apply(stemming)
train.head(10)

,user.id,age,text,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,...,Religion,Science,Sports-Recreation,Student,Technology,Telecommunications,Tourism,Transportation,indUnk,Count
0,1,17,dorito fuel space urllink : ian myer nasa new ...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,1.347240e-05
1,3,16,"yeah, set blog heather would happy...sh better...",0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,3.476749e-06
2,4,17,"much excit happen weekend, i'v realli busy. i'...",0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001,0.0,0.000,0.0,0.0,0.000,4.345937e-07
3,5,47,european pilgrimag high school student st. mic...,0.001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,9.561060e-06
4,6,16,last night dream blurry. detail remember: patt...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001,0.0,0.000,0.0,0.0,0.000,2.877010e-04
5,7,25,"june 21, 2004 spain v portug good euro portug ...",0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,1.303781e-06
6,12,23,"well, 1 day left... less 36 hour presenc favor...",0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,4.345937e-06
7,14,37,everi see post one forum goe someth like this....,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.001,0.0,0.0,0.000,4.172099e-05
8,15,17,well around 3 adam jake went outsid smoke pipe...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,1.173403e-05
9,16,15,"yesterday,w wif sister meet huiqing..r quit en...",0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001,0.0,0.000,0.0,0.0,0.000,1.477618e-05


In [18]:
train.to_csv("train_clean.csv", encoding = 'utf-8', index = False)

In [19]:
train=train.drop('Count',axis=1)

In [20]:
#new
count_vect = CountVectorizer(max_features=1000)
X_train_counts = count_vect.fit_transform(train.text)
X_train_counts.shape

(12880, 1000)

In [21]:
#new
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(12880, 1000)

In [22]:
train=train.drop(['user.id','text'],axis=1)

In [23]:
tfidf_df = pd.DataFrame(X_train_tfidf.todense())

In [24]:
train_total=train.join(tfidf_df,how='right')

In [25]:
train_total.head()

,age,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,BusinessServices,...,990,991,992,993,994,995,996,997,998,999
0,17,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.022685,0.032786,0.000000,0.021022,0.000000,0.000000,0.000000,0.052216,0.000000
1,16,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.067231,0.012146,0.018458,0.031151,0.047961,0.028057,0.000000,0.025792,0.000000
2,17,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.037379,0.000000,0.000000,0.049200,0.000000,0.000000,0.000000,0.182260
3,47,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.073518,0.024828,0.000000,0.000000,0.004717,0.026491,0.017346,0.006638
4,16,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000975,0.042241,0.031318,0.006025,0.028977,0.013567,0.018773,0.004285,0.105228,0.006443


In [26]:
train_age=train_total['age']

In [27]:
train_total=train_total.drop('age',axis=1)

In [28]:
clf = MultinomialNB().fit(train_total, train_age)

In [31]:
test=pd.read_csv('test.csv')

In [32]:
test_topic=test[['user.id','topic']]
test_topic=pd.concat([test_topic,pd.get_dummies(test_topic['topic'])],axis=1)
test_topic=test_topic.groupby('user.id').sum().reset_index()
test_id=test_topic['user.id']
test_count=test.groupby(['user.id'])["user.id"].size().reset_index(name='Count')
test_topic=pd.merge(test_topic,test_count,how='left',on='user.id')
test_merge=test_topic.iloc[:,1:41].div(test_topic.iloc[:,-1],axis=0)
test_merge=test_merge.iloc[:,0:41].div(1000,axis=0)
test_count=test_count['Count']
test_count=test_count.div(test_count.max(),axis=0)
test_merge=test_merge.join(test_id,how='right')
test_merge=test_merge.join(test_count,how='left')
test=test.groupby('user.id')["text"].apply(lambda x: "%s" % ' '.join(x)).reset_index(name='text')
test=pd.merge(test,test_merge,how='left',on='user.id')
test.head()

,user.id,text,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,...,Religion,Science,Sports-Recreation,Student,Technology,Telecommunications,Tourism,Transportation,indUnk,Count
0,2,I finally saw Nathan's tattoo tonig...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.001658
1,8,Today was the last day and the shor...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001,0.016347
2,9,What the hell am I doing on the int...,0.0,0.0,0.0,0.0,0.001,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.085525
3,10,the hotel&nbsp; urlLink urlLink ...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001,0.005212
4,11,Last night I received message fro...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000948


In [33]:
test['text'] = test['text'].apply(stopwords)
test['text'] = test['text'].apply(stemming)
test.head(10)

,user.id,text,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,...,Religion,Science,Sports-Recreation,Student,Technology,Telecommunications,Tourism,Transportation,indUnk,Count
0,2,final saw nathan tattoo tonight. like it! fact...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.001658
1,8,today last day shortest. one session last 10:0...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,0.016347
2,9,hell internet? 2:16am song: exact i'm at - wee...,0.0,0.0,0.0,0.0,0.001,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.085525
3,10,hotel&nbsp; urllink urllink thrid & win goal&n...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,0.005212
4,11,last night receiv messag friend minnesota. tol...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000948
5,13,"hey every1 came back school fun, think subject...",0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001,0.0,0.000,0.0,0.0,0.000,0.002132
6,20,chang blogspot address...mi origin reason want...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.001,0.0,0.0,0.000,0.034352
7,26,m33ting chrome sunlight > line say all. online...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001,0.0,0.000,0.0,0.0,0.000,0.030088
8,27,i'm mental prepar journi western psyche. told ...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001,0.0,0.000,0.0,0.0,0.000,0.000711
9,29,"hrm. i'v decid mu calori intake, up amount exc...",0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,0.000948


In [34]:
X_test_counts = count_vect.fit_transform(test.text)
X_test_counts.shape
tfidf_transformer = TfidfTransformer()
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)
X_test_tfidf.shape

(6440, 1000)

In [35]:
test.to_csv("test_clean.csv", encoding = 'utf-8', index = False)

In [36]:
test=test.drop('Count',axis=1)

In [37]:
test_2=test

In [38]:
test=test.drop(['user.id','text'],axis=1)
tfidf_df = pd.DataFrame(X_test_tfidf.todense())
test_total=test.join(tfidf_df,how='right')
test_total.head()

,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,BusinessServices,Chemicals,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.076234,0.057982,0.048750,0.000000,0.087800,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.038531,0.004509,0.003791,0.015468,0.003414,0.012808,0.012478,0.000000
2,0.0,0.0,0.0,0.0,0.001,0.0,0.0,0.0,0.0,0.0,...,0.0,0.075583,0.023179,0.005796,0.022742,0.010357,0.021942,0.001372,0.084883,0.006794
3,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.030119,0.034362,0.019261,0.000000,0.052034,0.000000,0.007925,0.000000
4,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.031186,0.000000,0.000000,0.000000,0.035918,0.000000,0.065645,0.000000


In [39]:
predicted = clf.predict(test_total)

In [40]:
cols = ["user.id", "age"]
test_output = pd.DataFrame(columns = cols)
test_output[["user.id"]] = test_2[["user.id"]]
test_output[["age"]] = predicted

In [41]:
test_output.head()

,user.id,age
0,2,23
1,8,23
2,9,23
3,10,24
4,11,23


In [42]:
# train.to_csv("train_FRE.csv", encoding = 'utf-8', index = False)
test_output.to_csv("test_sklearn_output.csv", encoding = 'utf-8', index = False)

In [ ]:
#def length(text):    
 #   '''a function which returns the length of text'''
 #   return len(text)

In [ ]:
#train['length'] = train['text'].apply(length)
#train.head(10)

In [ ]:
# create the object of tfid vectorizer
tfid_vectorizer = TfidfVectorizer("english")
# fit the vectorizer using the text data
tfid_vectorizer.fit(train['text'])
# collect the vocabulary items used in the vectorizer
dictionary = tfid_vectorizer.vocabulary_.items()  

In [ ]:
train.to_csv("train_sklearn_intermediate.csv", encoding = 'utf-8', index = False)

In [ ]:
tfid_matrix = tfid_vectorizer.transform(train['text'])
#collect the tfid matrix in numpy array

In [ ]:
tfid_matrix